In [1]:
import os
import random
import numpy as np
import sys
import torch
import pprint

root = '/'
pprint.pprint(sys.path)

sys.path.append(import_path)
pprint.pprint(sys.path)

['/opt/conda/lib/python310.zip',
 '/opt/conda/lib/python3.10',
 '/opt/conda/lib/python3.10/lib-dynload',
 '',
 '/opt/conda/lib/python3.10/site-packages']
['/opt/conda/lib/python310.zip',
 '/opt/conda/lib/python3.10',
 '/opt/conda/lib/python3.10/lib-dynload',
 '',
 '/opt/conda/lib/python3.10/site-packages',
 '/torch_cuda/pigvae_all']
['/opt/conda/lib/python310.zip',
 '/opt/conda/lib/python3.10',
 '/opt/conda/lib/python3.10/lib-dynload',
 '',
 '/opt/conda/lib/python3.10/site-packages',
 '/torch_cuda/pigvae_all',
 '/torch_cuda/ddpm-torch']


In [ ]:
import os
import sys
import pprint

import_path = root + 'pigvae_all'
sys.path.append(import_path)
pprint.pprint(sys.path)

import_path1 = root + 'pigvae_all/pigvae'
sys.path.append(import_path1)
pprint.pprint(sys.path)

In [2]:
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data.distributed import DistributedSampler
import random
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
import networkx as nx
from networkx.algorithms.shortest_paths.dense import floyd_warshall_numpy

from networkx.generators.random_graphs import *
from networkx.generators.ego import ego_graph
from networkx.generators.geometric import random_geometric_graph

/opt/conda/lib/python3.10/site-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /opt/conda/lib/python3.10/site-packages/torch_scatter/_version_cpu.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSsb
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/opt/conda/lib/python3.10/site-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /opt/conda/lib/python3.10/site-packages/torch_cluster/_version_cpu.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSsb
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
/opt/conda/lib/python3.10/site-packages/torch_geometric/typing.py:113: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: /opt/conda/lib/python3.10/site-packages/torch_spline_conv/_version_cpu.so: undefined symbol: _ZN

In [3]:
num_node_f = 43
num_edge_f = 6

#from ZINC model

In [4]:
hparams = {
    "vae":True,
    "kld_loss_scale":0.01,
    "perm_loss_scale":0.1,
    "property_loss_scale":0.5,
    "num_node_features":num_node_f,
    "num_edge_features":6+num_edge_f+1,
    "emb_dim": 90,
    'graph_encoder_hidden_dim': 256,
    'graph_encoder_k_dim': 64,
    'graph_encoder_v_dim': 64,
    'graph_encoder_num_heads': 16,
    'graph_encoder_ppf_hidden_dim': 512,
    'graph_encoder_num_layers': 16,
    'graph_decoder_hidden_dim': 256,
    'graph_decoder_k_dim': 64,
    'graph_decoder_v_dim': 64,
    'graph_decoder_num_heads': 16,
    'graph_decoder_ppf_hidden_dim': 512,
    'graph_decoder_num_layers': 16,
    "graph_decoder_pos_emb_dim": 64,
    'property_predictor_hidden_dim': 3,
    'num_properties': 1
}

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [6]:
from torch import nn
from torch.nn import Linear, LayerNorm, Dropout
from torch.nn.functional import relu, pad
from pigvae.graph_transformer import Transformer, PositionalEncoding
#from pigvae.synthetic_graphs.data import DenseGraphBatch

from pigvae.models import GraphEncoder, GraphDecoder, Permuter

In [7]:
from torch import nn
from torch.nn import Linear, LayerNorm, Dropout
from torch.nn.functional import relu, pad
from pigvae.graph_transformer import Transformer, PositionalEncoding
from pigvae.models import GraphAE

model = GraphAE(hparams).to(device)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
load_model_dir = root + 'save_models/zinc/pig-beta-vae_models/'
load_model_path = load_model_dir + "pigvae_best_model.pt"

model = torch.load(load_model_path)

# Loop

In [9]:
def load_pickes(load_dir):
    with open(load_dir + "node_features.pickle", 'br') as fa:
        node_features = pickle.load(fa)
        fa.close
          
    with open(load_dir + "edge_features.pickle", 'br') as fb:
        edge_features = pickle.load(fb)
        fb.close
          
    with open(load_dir + "masks.pickle", 'br') as fc:
        masks = pickle.load(fc)
        fc.close

    with open(load_dir + "props.pickle", 'br') as fd:
        props = pickle.load(fd)
        fd.close

    with open(load_dir + "targets.pickle", 'br') as fe:
        targets = pickle.load(fe)
        fe.close
        
    return node_features, edge_features, masks, props, targets

In [10]:
def graphs_to_embs(dataloader):
    
    z_mus = []
    t_list = []
    
    for batch_idx, batch_data in enumerate(dataloader):

        batch_stds = []
        
        if batch_idx % 100 == 0 :
            print(batch_idx)

        node_features, edge_features, mask, props, targets = batch_data
        node_features, edge_features, mask, props = node_features.to(device), edge_features.to(device), mask.to(device), props.to(device)
    
        z, _, _, _ = model.encode(node_features, edge_features, mask)

        z_mus.extend(z.cpu().detach().numpy())
        t_list.extend(targets.cpu().detach().numpy())
    
        del z
        torch.cuda.empty_cache()
        
    return np.array(z_mus), np.array(t_list)

In [11]:
task_names = ["bace", "ctsd", "mmp2", "malaria", "esol", "freesolv", "lipo"]

In [12]:
import pickle

for task in task_names:
    
    load_dir = root + "dataset/eval_dataset/molecular_properties/zinc/graphs/" + task  + "/"
    sample_save_dir = load_model_dir + "samples_for_mcmc/" + task + "/"
    
    print(sample_save_dir)

    if not os.path.exists(sample_save_dir):
        os.makedirs(sample_save_dir)
        
    node_features, edge_features, masks, props, targets = load_pickes(load_dir)
    dataset = torch.utils.data.TensorDataset(node_features, edge_features, masks, props, targets)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=20, shuffle=False)
    
    z_mus, ys = graphs_to_embs(dataloader)
    print(z_mus.shape, ys.shape)
    
    np.save(sample_save_dir + "embs_mu.npy", z_mus)
    np.save(sample_save_dir + "targets.npy", ys)

/torch_cuda/save_models/pig-beta-vae_models/samples_for_mcmc/bace/
0
(502, 90) (502,)
/torch_cuda/save_models/pig-beta-vae_models/samples_for_mcmc/ctsd/
0
(84, 90) (84,)
/torch_cuda/save_models/pig-beta-vae_models/samples_for_mcmc/mmp2/
0
(1046, 90) (1046,)
/torch_cuda/save_models/pig-beta-vae_models/samples_for_mcmc/malaria/
0
100
(3019, 90) (3019,)
/torch_cuda/save_models/pig-beta-vae_models/samples_for_mcmc/esol/
0
(1080, 90) (1080,)
/torch_cuda/save_models/pig-beta-vae_models/samples_for_mcmc/freesolv/
0
(642, 90) (642,)
/torch_cuda/save_models/pig-beta-vae_models/samples_for_mcmc/lipo/
0
(1903, 90) (1903,)
/torch_cuda/save_models/pig-beta-vae_models/samples_for_mcmc/vp/
0
100
(2629, 90) (2629,)
/torch_cuda/save_models/pig-beta-vae_models/samples_for_mcmc/bp/
0
100
200
(5368, 90) (5368,)


In [13]:
import pickle

task = "logp"

load_dir = root + "dataset/eval_dataset/molecular_properties/zinc/graphs/" + task  + "/"
sample_save_dir = load_model_dir + "samples_for_mcmc/" + task + "/"

print(sample_save_dir)

if not os.path.exists(sample_save_dir):
    os.makedirs(sample_save_dir)

node_features, edge_features, masks, props, targets = load_pickes(load_dir)
dataset = torch.utils.data.TensorDataset(node_features, edge_features, masks, props, targets)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=False)

z_mus, ys = graphs_to_embs(dataloader)
print(z_mus.shape, ys.shape)

np.save(sample_save_dir + "embs_mu.npy", z_mus)
np.save(sample_save_dir + "targets.npy", ys)

/torch_cuda/save_models/pig-beta-vae_models/samples_for_mcmc/logp/
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
(100000, 90) (100000,)
